# Stadistic testing
In this notebooke we're going review some of the basics of statistical testing in python. We're going to talk about hypotesis testing, statistical significance, and using scipy to run student's t-tests.
We use statistics in a lot of different ways in data science, and on this lectura, I want to refresh you knowledge of hypothesis testing, which is a core data analysis activity behind experimentation. The goal of hypothesis is to determine if, for instance, the two different conditions we have in an experiment have resulted in different impacts.
Now, scipy is an interesting cikkectuib if libraries fo data science and you'll use most or perpahs all of these libraries. It incluedes numpy and pandas, but also plotting libraries such as matplotlib, and a number of scientific library functions as well.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

When we do hyppothesus testing, we actually have two staements of interest: The fist is our actual explanation, whcich we call the alternative hypothesis, and the second is that explanation we have is no sufficient, and we call this the null hypothesis. Our testing method is to determine whether the null hypothesis is true or not. If we find that there is a difference between groups, then we can reject the null hypothesis and we accept our alternative.

In [2]:
df = pd.read_csv('data/grades.csv')
df.head()

,student_id,assignment1_grade,assignment1_submission,assignment2_grade,assignment2_submission,assignment3_grade,assignment3_submission,assignment4_grade,assignment4_submission,assignment5_grade,assignment5_submission,assignment6_grade,assignment6_submission
0,B73F2C11-70F0-E37D-8B10-1D20AFED50B1,92.733946,2015-11-02 06:55:34.282000000,83.030552,2015-11-09 02:22:58.938000000,67.164441,2015-11-12 08:58:33.998000000,53.011553,2015-11-16 01:21:24.663000000,47.710398,2015-11-20 13:24:59.692000000,38.168318,2015-11-22 18:31:15.934000000
1,98A0FAE0-A19A-13D2-4BB5-CFBFD94031D1,86.790821,2015-11-29 14:57:44.429000000,86.290821,2015-12-06 17:41:18.449000000,69.772657,2015-12-10 08:54:55.904000000,55.098125,2015-12-13 17:32:30.941000000,49.588313,2015-12-19 23:26:39.285000000,44.629482,2015-12-21 17:07:24.275000000
2,D0F62040-CEB0-904C-F563-2F8620916C4E,85.512541,2016-01-09 05:36:02.389000000,85.512541,2016-01-09 06:39:44.416000000,68.410033,2016-01-15 20:22:45.882000000,54.728026,2016-01-11 12:41:50.749000000,49.255224,2016-01-11 17:31:12.489000000,44.329701,2016-01-17 16:24:42.765000000
3,FFDF2B2C-F514-EF7F-6538-A6A53518E9DC,86.030665,2016-04-30 06:50:39.801000000,68.824532,2016-04-30 17:20:38.727000000,61.942079,2016-05-12 07:47:16.326000000,49.553663,2016-05-07 16:09:20.485000000,49.553663,2016-05-24 12:51:18.016000000,44.598297,2016-05-26 08:09:12.058000000
4,5ECBEEB6-F1CE-80AE-3164-E45E99473FB4,64.813800,2015-12-13 17:06:10.750000000,51.491040,2015-12-14 12:25:12.056000000,41.932832,2015-12-29 14:25:22.594000000,36.929549,2015-12-28 01:29:55.901000000,33.236594,2015-12-29 14:46:06.628000000,33.236594,2016-01-05 01:06:59.546000000


In [3]:
#If we take a look the data frame inside, we see have six different assignments
print("There are {} rows and {} columns".format(df.shape[0], df.shape[1]))

There are 2315 rows and 13 columns


For purpose of this lecture, Let's segment this population into two pieces. Let's say those finis the fist assignment by the end of December 2015, we'll call them early finishers, and those who finish ir sometime after that, we'll call them late finishers.

In [5]:
early_finishers = df[pd.to_datetime(df['assignment1_submission']) < '2016']
early_finishers.head()
late_finishers = df[~df.index.isin(early_finishers.index)]
late_finishers.head()

,student_id,assignment1_grade,assignment1_submission,assignment2_grade,assignment2_submission,assignment3_grade,assignment3_submission,assignment4_grade,assignment4_submission,assignment5_grade,assignment5_submission,assignment6_grade,assignment6_submission
2,D0F62040-CEB0-904C-F563-2F8620916C4E,85.512541,2016-01-09 05:36:02.389000000,85.512541,2016-01-09 06:39:44.416000000,68.410033,2016-01-15 20:22:45.882000000,54.728026,2016-01-11 12:41:50.749000000,49.255224,2016-01-11 17:31:12.489000000,44.329701,2016-01-17 16:24:42.765000000
3,FFDF2B2C-F514-EF7F-6538-A6A53518E9DC,86.030665,2016-04-30 06:50:39.801000000,68.824532,2016-04-30 17:20:38.727000000,61.942079,2016-05-12 07:47:16.326000000,49.553663,2016-05-07 16:09:20.485000000,49.553663,2016-05-24 12:51:18.016000000,44.598297,2016-05-26 08:09:12.058000000
6,3217BE3F-E4B0-C3B6-9F64-462456819CE4,87.498744,2016-03-05 11:05:25.408000000,69.998995,2016-03-09 07:29:52.405000000,55.999196,2016-03-16 22:31:24.316000000,50.399276,2016-03-18 07:19:26.032000000,45.359349,2016-03-19 10:35:41.869000000,45.359349,2016-03-23 14:02:00.987000000
7,F1CB5AA1-B3DE-5460-FAFF-BE951FD38B5F,80.576090,2016-01-24 18:24:25.619000000,72.518481,2016-01-27 13:37:12.943000000,65.266633,2016-01-30 14:34:36.581000000,65.266633,2016-02-03 22:08:49.002000000,65.266633,2016-02-16 14:22:23.664000000,65.266633,2016-02-18 08:35:04.796000000
9,E2C617C2-4654-622C-AB50-1550C4BE42A0,59.270882,2016-03-06 12:06:26.185000000,59.270882,2016-03-13 02:07:25.289000000,53.343794,2016-03-17 07:30:09.241000000,53.343794,2016-03-20 21:45:56.229000000,42.675035,2016-03-27 15:55:04.414000000,38.407532,2016-03-30 20:33:13.554000000


There are a lotsof other ways to do this. For intance, you could just copy and paste the fisrt projection and change the signd from less than o greater tha or equal to. This is ok, but if you decide you want to change the date down the road you have to remember to change it in two places. You could also do a joi nof the dataframe df with early_finishers - if do you a lest join you only keep items in the left dataframe, so this would have been a old answer. You also could have written a function that determines if someone is early or late, and the called. apply() on the dataframe and added a new column to the dataframe. This is a pretty reasonable answer as well.

In [7]:
# As you've seen, the pandas data frame object has a variety of statistical function associated with it. If we call the mean function directly on the data frame, we see that each of the means 
# for the assignments are aclculated. Let's compare the means for our two populations
print(early_finishers['assignment1_grade'].mean())
print(late_finishers['assignment1_grade'].mean())

74.94728457024304
74.0450648477065


Ok, these look prety similar. But, are they the same? What do we mean by similar? This is where the students' t-test comes in. It allows us to form the alternative hypothesis ("These are different") as well as the null hypothesis ("These are the same") and then test that null hypothesis. When doing hypothesis testing, we have to choose a significance level as a threshold for how much of a chance we're willing to accept. This significance level is typically called alpha. For this example, Let's use a threshold of 0.05 for our alpha or 5%. Now this is a commonly used number but it's really quite arbitrary.
The SciPy library contains a number of different statistical tests and forms a basis for hypothesis testing in Python and we're going to use the ttest_ind() function which does an independent t-test (meaning the populations are not relaed to one another). The result of ttest_index() are the t-statistic and a p-value. It's this latter value, the probability, which is most important to us, as it indicates the chance (between 0 and 1) of our null hypothesis being True.

In [8]:
from scipy.stats import ttest_ind
ttest_ind(early_finishers['assignment1_grade'], late_finishers['assignment1_grade'])
# So here we see that the probability is 0.18, and this is above our alpha value of 0.05. This means that we cannot reject the null hypothesis. The null hypothesis was that the two populations
# are the same, and we don't have enough certainty in our evidence (because it is greater tha alpha) to come to a conclusion to the contrary. This doesn't mean that we have proven the populations
# are the same.

Ttest_indResult(statistic=1.3223540853721596, pvalue=0.18618101101713855)

In [10]:
# Why don't we check the other assignment grades?
print(ttest_ind(early_finishers['assignment2_grade'], late_finishers['assignment2_grade']))
print(ttest_ind(early_finishers['assignment3_grade'], late_finishers['assignment3_grade']))
print(ttest_ind(early_finishers['assignment4_grade'], late_finishers['assignment4_grade']))
print(ttest_ind(early_finishers['assignment5_grade'], late_finishers['assignment5_grade']))
print(ttest_ind(early_finishers['assignment6_grade'], late_finishers['assignment6_grade']))

Ttest_indResult(statistic=1.2514717608216366, pvalue=0.2108889627004424)
Ttest_indResult(statistic=1.6133726558705392, pvalue=0.10679998102227865)
Ttest_indResult(statistic=0.049671157386456125, pvalue=0.960388729789337)
Ttest_indResult(statistic=-0.05279315545404755, pvalue=0.9579012739746492)
Ttest_indResult(statistic=-0.11609743352612056, pvalue=0.9075854011989656)


Ok, so it looks like in this data we do not have enough evidence to suggest the populations differ with respect to grade. Let's take a look those p-values for a moment though, because they are saying things that can inform experimental design down the road. For instance, one of the assignments, assignment 3, has a p-value around 0.1. This means that if we accepted a level of chance similarity of 11% this would have been considered statistically significant. As a research, this would suggest to me that there is somenthing here worth considering following up on. For instance, if we had a small number of participants (we don't) or if there was something unique about this assignment as it relates to our experiment (whatever it was) then there may be followup experiments we could run.
P-values have come under fire recently for being induficient for telling us enough about the interactions which are happening, and two other techniques, confidence intervalues and bayesian analyses, are being used more regularly. One issue with p-values is that as you run more tests are likely to get a value which is statistically significant just by chance.

In [14]:
df1 = pd.DataFrame([np.random.random(100) for x in range(100)])
# Pause this and reflect -- do you understand the list pomprehension and how I created this DataFrame? You don't have to use a list comprehension to do this, but you should be able to read this
# and figure out how it works as this a commonly used approach on web forums.
df2 = pd.DataFrame([np.random.random(100) for x in range(100)])
df1.head()
# Are these two DataFrames the same? Maybe a better question is, for a given row inside of df1, is it the same as the row inside df2?. Let's take a look. Let's say our critical value is 0.1
# or and alpha of 10%. And we're going to compare each column in df1 to the same numbered column in df2. And we'll report when the p-value isn't less than 10% which means that we have sufficient
# evidence to say that the columns are different.

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.906527,0.122048,0.667143,0.498016,0.699089,0.386407,0.205689,0.356401,0.906706,0.870242,...,0.567573,0.422556,0.910511,0.376145,0.785454,0.002866,0.312443,0.608675,0.807268,0.975747
1,0.369832,0.493761,0.308976,0.716444,0.059590,0.026350,0.381108,0.060707,0.703992,0.640314,...,0.394702,0.125042,0.149529,0.436992,0.941209,0.413437,0.727757,0.119147,0.497130,0.853357
2,0.992802,0.170454,0.679962,0.768920,0.134241,0.784590,0.186404,0.147237,0.697916,0.108090,...,0.521540,0.955655,0.151910,0.202401,0.137684,0.583242,0.288515,0.152871,0.020032,0.773015
3,0.512483,0.976235,0.523540,0.846988,0.806182,0.799384,0.053837,0.850632,0.639410,0.463102,...,0.507921,0.527972,0.951124,0.065320,0.347305,0.925234,0.749949,0.810437,0.549980,0.585343
4,0.421669,0.022637,0.806027,0.687236,0.311395,0.235594,0.506871,0.556030,0.069906,0.154972,...,0.316966,0.176643,0.152680,0.896096,0.447172,0.853328,0.686754,0.593648,0.312059,0.012488


In [21]:
def test_columns(alpha=0.1):
    num_diff = 0
    for col in df1.columns:
        teststat, pval = ttest_ind(df1[col], df2[col])
        if pval <= alpha:
            print("Col {} is stadistically significantly different at alpha = {}, pval = {}". format(col, alpha, pval))
            num_diff = num_diff + 1
    print("Total number different was {}, which is {}%".format(num_diff, float(num_diff)/len(df1.columns)*100))

test_columns()
# Interesting, so we see that there are a bunch of columns that are different! IN fact, that number looks a lot like the alpha value we chose. So what's going on - shouldn't all of the columns be
# the same? Remember that all the ttest does is check if two sets are similar given some level of confidence, in our case, 10%. The more random comparisons you do, the more will just happen to be
# the same by chance. In this example, we checked 100 columns, so we would expect there to be roughly 10 of them if our alpha was 0.1.
test_columns(0.05)
# So, keep this in mind when yu are doing statistical tests like the t-test which has a p-value. Understand that this p-value isn't magic, that it's a threshold for you when reporting results
# and trying to answer your hypothesis. Whta's a reasonalbe threshold? Depends on your question, and you need to engage domain experts to better understand what they would consider significant.

Col 6 is stadistically significantly different at alpha = 0.1, pval = 0.05337930970728126
Col 9 is stadistically significantly different at alpha = 0.1, pval = 0.014493883127167092
Col 21 is stadistically significantly different at alpha = 0.1, pval = 0.08684051920560812
Col 28 is stadistically significantly different at alpha = 0.1, pval = 0.0010143415081508669
Col 29 is stadistically significantly different at alpha = 0.1, pval = 0.07754102377783406
Col 52 is stadistically significantly different at alpha = 0.1, pval = 0.0632418084059522
Col 62 is stadistically significantly different at alpha = 0.1, pval = 0.06194914868615695
Col 75 is stadistically significantly different at alpha = 0.1, pval = 0.09382263500873238
Col 81 is stadistically significantly different at alpha = 0.1, pval = 0.03770836837891057
Total number different was 9, which is 9.0%
Col 9 is stadistically significantly different at alpha = 0.05, pval = 0.014493883127167092
Col 28 is stadistically significantly differe